In [1]:
import random
import gym
import numpy as np
import random
import pandas as pd
import torch
import torch.nn as nn
from copy import deepcopy
import torch.optim as optim
from collections import deque
from sklearn.preprocessing import StandardScaler
import random
import plotly.express as px
# Normalize features
from sklearn.preprocessing import StandardScaler
from collections import deque
import torch.nn.functional as F
import random
import gym
import numpy as np
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('/kaggle/input/eqqerff/preprocessed.csv')
data.drop(columns= ['Unnamed: 0','index'], inplace = True)
# preprocessing
data['DATETIME'] = pd.to_datetime(data['DATETIME'])
data['TIME'] = data['TIME'].astype(str).str.rstrip('000')
data['hour'] = data['DATETIME'].dt.hour
data['minute'] = data['DATETIME'].dt.minute
data['day_of_week'] = data['DATETIME'].dt.day_name()
data['second'] =  data['DATETIME'].dt.second
day_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
data['day_of_week'] = data['day_of_week'].map(day_map)
data.drop(columns= ['DATETIME','TIME'], inplace = True) 
data.drop(columns= ['TRADE_PRICE','TRADENO'], inplace = True)
data= data.dropna()
data.drop(columns = ['DATE', 'ACTION','ORDERNO','TRADE_TIME'], inplace = True)
# Create a dictionary to map string values to numeric values
buysell_map = {'B': 0, 'S': 1}

# Map the string values to numeric values
data['BUYSELL'] = data['BUYSELL'].map(buysell_map)
data.drop(columns = 'TRADEPRICE', inplace = True)

In [3]:
from tqdm import tqdm
# Normalizing the data
def normalize_data(df):
    df_normalized = df.copy()
    for column in df.columns:
        if df[column].dtype == np.float64 or df[column].dtype == np.int64:
            df_normalized[column] = (df[column] - df[column].mean()) / df[column].std()
    return df_normalized

data = normalize_data(data)

In [4]:
#def reward_function(df):
 #   # Calculate best bid-ask spread for each date
  #  best_spread_per_date = df.groupby('DATE').apply(lambda x: x['BID_PRICE'].max() - x['ASK_PRICE'].min())

    #rewards = []
    #for index, row in df.iterrows():
        # Calculate execution price deviation from best bid-ask spread
       # execution_price_deviation = row['PRICE'] - best_spread_per_date[row['DATE']]

        # Calculate execution speed penalty
       # execution_speed_penalty = 100 / 1000  # converting milliseconds to seconds

      #  # Calculate commission cost
       # commission_cost = 0.01 * row['TRADE_VOLUME']
       # # Calculate market impact penalty (quadratic function)
       # market_impact_penalty = 0.0001 * (row['TRADE_VOLUME'] ** 2)

        # Calculate total reward
        #reward = -execution_price_deviation - execution_speed_penalty -  commission_cost- market_impact_penalty
        #rewards.append(reward)

   # return rewards
#
# rewards = reward_function(data)

In [5]:
class TradingEnv:
    def __init__(self, data, ac_dict=None, ac_type='vanilla6'):
        self.data = data
        self.current_step = 0
        self.done = False

        # action space: 0 for hold, 1 for buy
        self.action_space = [0, 1]

        # observation space: based on the number of columns in data
        self.observation_space = len(data.columns)

        # variables specific to your environment
        self.inventory = 0  # Starting with zero inventory
        self.time_points_for_execution = 100
        self.end_time = len(data)
        self.real_time = 0
        self.state = self.reset()

        # Metrics tracking
        self.cumulative_reward = 0
        self.cumulative_rewards = []
        self.implementation_shortfall = []

        # action dictionary and type
        self.ac_dict = ac_dict if ac_dict is not None else {0: 'hold', 1: 'buy'}
        self.ac_type = ac_type

    def reset(self):
        self.current_step = 0
        self.done = False
        self.inventory = 0
        self.cumulative_reward = 0
        self.cumulative_rewards = []
        self.implementation_shortfall = []

        self.state = {
            'price': self.data['PRICE'].iloc[0],
            'inventory': self.inventory,
            'time': 0
        }
        self.real_time = 0
        return self.state

    def _next_observation(self):
        # Return current observation based on the data at current_step and include inventory
        obs = self.data.iloc[self.current_step].to_dict()
        obs['inventory'] = self.state['inventory']
        return obs

    def get_price(self):
        return self.data['PRICE'].iloc[self.current_step]

    def is_done(self):
        return self.current_step >= len(self.data) - 1

    def step(self, action):
        current_price = self.get_price()
        price_penalty = max(0, current_price - self.state['price']) * action

        reward = self.state['price'] * action - 2.5 * (action ** 2) - price_penalty

        self.state['time'] = round(self.state['time'] + 1 / (self.time_points_for_execution - 1), 2)
        self.real_time = (self.real_time + 1) if (self.real_time + 1) % self.end_time != 0 else 0
        self.state['inventory'] = round(self.state['inventory'] - action, 2)
        self.state['price'] = current_price  # Update state price to current price

        self.current_step += 1
        self.cumulative_reward += reward
        self.cumulative_rewards.append(self.cumulative_reward)

        # Calculate implementation shortfall
        initial_price = self.data['PRICE'].iloc[0]
        isf = (initial_price - current_price) * action
        self.implementation_shortfall.append(isf)

        return self._next_observation(), reward, self.is_done()

    def render(self):
        # Implement rendering if needed for visualization
        pass
    
    def get_current_price(self):
        # return the current price from the data
        return self.data.iloc[self.current_step]['PRICE']
    
    def get_best_price(self, action):
        # return the best price from bid and ask prices
        bid_price = self.data.iloc[self.current_step]['BID_PRICE']
        ask_price = self.data.iloc[self.current_step]['ASK_PRICE']
        return bid_price if action == 1 else ask_price

    

    def plot_cumulative_rewards(self):
        fig = px.line(
            x=range(len(self.cumulative_rewards)),
            y=self.cumulative_rewards,
            labels={'x': 'Steps', 'y': 'Cumulative Reward'},
            title='Cumulative Reward over Episodes'
        )
        fig.show()

    def plot_implementation_shortfall(self):
        fig = px.line(
            x=range(len(self.implementation_shortfall)),
            y=self.implementation_shortfall,
            labels={'x': 'Steps', 'y': 'Implementation Shortfall'},
            title='Implementation Shortfall over Episodes'
        )
        fig.show()

    def calculate_parameters(self):
        prices = self.data['PRICE']
        volumes = self.data['VOLUME']

        # sigma (volatility)
        returns = prices.pct_change().dropna()
        sigma = returns.std()

        # tau (average time between trades in seconds)
        time_diffs = self.data['second'].diff().dropna()
        tau = time_diffs.mean()

        #  eta and rho based on a more detailed market impact model
        eta = self.calculate_eta()
        rho = self.calculate_rho()

        # a default value for lambda (risk aversion parameter)
        lamb = 1e-6

        # kappa
        kappa = np.sqrt(abs(lamb * sigma**2 / (eta * (1 - rho * tau / (2 * eta)))))

        return eta, rho, sigma, tau, lamb, kappa

    def calculate_eta(self):
        # the market impact cost per unit traded
        price_impact = self.data['PRICE'] - self.data['MID_PRICE']
        volume = self.data['VOLUME']

        # Filter out zero volumes to avoid division by zero
        valid_indices = volume > 0
        price_impact = price_impact[valid_indices]
        volume = volume[valid_indices]

        # the market impact per unit volume traded
        market_impact_per_volume = price_impact / volume
        eta = market_impact_per_volume.mean()  # Average market impact per unit volume

        return eta

    def calculate_rho(self):
        # the standard deviation of returns as a proxy for risk
        returns = self.data['PRICE'].pct_change().dropna()
        return_std = returns.std()

        # Risk aversion might be proportional to return standard deviation
        rho = return_std * 1e-2 

        return rho

class AlmgrenChrissAgent:
    def __init__(self, env, time_horizon, eta, rho, sigma, tau, lamb, kappa):
        self.ac_dict = deepcopy(env.ac_dict)
        self.ac_type = deepcopy(env.ac_type)
        self.eta = eta
        self.rho = rho
        self.sigma = sigma
        self.tau = tau
        self.time_horizon = time_horizon
        self.steps, self.j = time_horizon / self.tau, 1
        self.kappa = kappa

    def reset(self):
        self.j = 1

    def act(self, state):
        def closest_action(nj):
            action = 1 if nj >= 0.5 else 0
            return action

        if self.ac_type in ['vanilla6', 'vanilla20']:
            inventory = state['inventory']  
            if inventory == 0:
                # Handle division by zero scenario
                nj = 0  # or any other appropriate action or value
            elif self.kappa == 0:
                nj = self.tau / self.time_horizon * (1 / inventory)
            else:
                nj = 2 * np.sinh(0.5 * self.kappa * self.tau) * np.cosh(self.kappa * (self.time_horizon - (self.j - 0.5) * self.tau)) * (1 / inventory) / np.sinh(self.kappa * self.time_horizon)
            self.j += 1
            if self.j == self.steps + 1:
                nj = 1
            action = closest_action(nj)

            return action
        elif self.ac_type == 'prop_of_ac':
            action = closest_action(1)
            return action
        else:
            raise Exception('Unknown Action Type')

        return action


class RandomAgent:
    def __init__(self, env):
        self.action_space = env.action_space

    def act(self, state):
        return np.random.choice(self.action_space)

class GreedyAgent:
    def __init__(self, env):
        self.action_space = env.action_space

    def act(self, state):
        return 1  # Always buy

def evaluate_agent(agent, env, num_episodes=10):
    episode_rewards = []
    episode_implementation_shortfalls = []
    for _ in range(num_episodes):
        obs = env.reset()
        episode_reward = 0
        while not env.is_done():
            action = agent.act(obs)
            obs, reward, done = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
        episode_implementation_shortfalls.append(sum(env.implementation_shortfall))
    return episode_rewards, episode_implementation_shortfalls


# Initialize environment
env = TradingEnv(data)
eta, rho, sigma, tau, lamb, kappa = env.calculate_parameters()

# Initialize agents
agents = [
    AlmgrenChrissAgent(env, len(data), eta, rho, sigma, tau, lamb, kappa),
    RandomAgent(env),
    GreedyAgent(env)
]

# Evaluate agents
agent_rewards = {}
agent_implementation_shortfalls = {}
for agent in agents:
    agent_name = agent.__class__.__name__
    rewards, implementation_shortfalls = evaluate_agent(agent, env)
    agent_rewards[agent_name] = rewards
    agent_implementation_shortfalls[agent_name] = implementation_shortfalls

# Print and compare results
for agent_name, rewards in agent_rewards.items():
    print(f"{agent_name}: Average Cumulative Reward = {np.mean(rewards)}")
for agent_name, isf in agent_implementation_shortfalls.items():
    print(f"{agent_name}: Average Implementation Shortfall = {np.mean(isf)}")

# Plot cumulative rewards using Plotly
for agent_name, rewards in agent_rewards.items():
    fig = px.line(
        x=range(len(rewards)),
        y=np.cumsum(rewards),
        labels={'x': 'Episodes', 'y': 'Cumulative Reward'},
        title=f'Cumulative Rewards Comparison - {agent_name}'
    )
    fig.show()

# Plot implementation shortfall using Plotly
for agent_name, isf in agent_implementation_shortfalls.items():
    fig = px.line(
        x=range(len(isf)),
        y=np.cumsum(isf),
        labels={'x': 'Episodes', 'y': 'Implementation Shortfall'},
        title=f'Implementation Shortfall Comparison - {agent_name}'
    )
    fig.show()

AlmgrenChrissAgent: Average Cumulative Reward = 0.0
RandomAgent: Average Cumulative Reward = -423836.82489566505
GreedyAgent: Average Cumulative Reward = -847787.4698382231
AlmgrenChrissAgent: Average Implementation Shortfall = 0.0
RandomAgent: Average Implementation Shortfall = -57323.31361141176
GreedyAgent: Average Implementation Shortfall = -114400.11528077922


### Average Cumulative Reward:

1. **AlmgrenChrissAgent**: 15.90440891642201
   - This agent has an average cumulative reward of approximately 15.9. Cumulative reward typically measures the total reward accumulated over multiple episodes or time periods. In this case, the AlmgrenChrissAgent achieved a modest cumulative reward, suggesting its trading strategy led to a relatively stable or conservative performance in terms of reward generation.

2. **RandomAgent**: 6425.041818000086
   - The RandomAgent has a significantly higher average cumulative reward compared to the AlmgrenChrissAgent. This suggests that the RandomAgent, which likely makes random or stochastic decisions without a specific strategy, sometimes performs well due to chance. However, the high variance in rewards indicates inconsistency in its performance.

3. **GreedyAgent**: 12663.166548637077
   - The GreedyAgent outperforms both the AlmgrenChrissAgent and the RandomAgent significantly in terms of average cumulative reward. A greedy strategy often focuses on maximizing immediate gains without considering long-term consequences. The high cumulative reward suggests that the GreedyAgent's strategy, while aggressive, yielded substantial returns in the simulated trading environment.

### Average Implementation Shortfall:

1. **AlmgrenChrissAgent**: 0.0
   - The AlmgrenChrissAgent has an average implementation shortfall of 0.0, indicating that it managed to execute trades at prices close to the best available prices (bid-ask spread). This suggests that the agent's strategy is effective in minimizing the cost associated with executing trades in terms of price deviation from optimal prices.

2. **RandomAgent**: 574.7519118496991
   - The RandomAgent has an average implementation shortfall of approximately 574.75. This higher value indicates that the agent often faced significant costs associated with executing trades, possibly due to random or suboptimal decision-making.

3. **GreedyAgent**: 1164.8528570058172
   - The GreedyAgent has the highest average implementation shortfall among the three agents, indicating that its aggressive strategy led to substantial costs associated with trade execution. The higher shortfall suggests that the agent's approach resulted in more significant price deviations and higher trading costs.


In summary, while the GreedyAgent achieved the highest average cumulative reward, it also incurred the highest implementation shortfall, indicating higher trading costs. The AlmgrenChrissAgent, despite having a lower cumulative reward, managed to execute trades with minimal price deviation (zero shortfall), reflecting efficient trade execution. The RandomAgent shows a wide variance in performance, with moderate cumulative reward and implementation shortfall, highlighting the unpredictability of random decision-making in trading scenarios. These metrics collectively provide insights into each agent's trading strategy's strengths and weaknesses in your simulated environment.